# 006_HANSOL_0Act_Del_R
## 한솔 인티큐브 콜센터 데이터
 
## 데이터 처리 과정
### 005_HANSOL_All_Act_R에 2 - 3부터 시작
### 4. Acticity 생성
### 5. Activity Time이 0초인 경우 제외
### 6. 005와 같음

In [1]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
√ ggplot2 2.2.1     √ purrr   0.2.4
√ tibble  1.4.1     √ dplyr   0.7.4
√ tidyr   0.7.2     √ stringr 1.2.0
√ readr   1.1.1     √ forcats 0.2.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
library(plyr)

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: 'plyr'

The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following object is masked from 'package:purrr':

    compact



In [3]:
library(reshape2)


Attaching package: 'reshape2'

The following object is masked from 'package:tidyr':

    smiths



In [6]:
hans <- read.csv("C:\\Users\\진수빈\\hansoltime.csv")
head(hans)

CALLID,ACWTIME,ANSHOLDTIME,CONSULTTIME,DISPTIME,DURATION,SEGSTART,SEGSTART_UTC,SEGSTOP,SEGSTOP_UTC,...,ORIGLOGIN,ANSLOGIN,LASTOBSERVER,LASTDIGITS,CALLING_II,VDN2,VDN3,ASAI_UUI,AGENTSURPLUS,AGENTSKILLLEVEL
2340247,0,15,0,21,629,21.12.2016 02:54:52,2016-12-20 17:54:52,2016-12-21 03:05:21,2016-12-20 18:05:21,...,NA,200508,NA,1,NA,510211,NA,,2,4
2340254,0,0,0,2,93,21.12.2016 03:04:59,2016-12-20 18:04:59,2016-12-21 03:06:32,2016-12-20 18:06:32,...,NA,200591,NA,,NA,NA,NA,,2,4
2340255,0,0,2,1,31,21.12.2016 03:06:18,2016-12-20 18:06:18,2016-12-21 03:06:49,2016-12-20 18:06:49,...,200431,NA,NA,,NA,NA,NA,,0,0
2340259,1,11,0,8,122,21.12.2016 03:04:25,2016-12-20 18:04:25,2016-12-21 03:07:12,2016-12-20 18:07:12,...,NA,200331,NA,,NA,NA,NA,,1,4
2340260,0,0,0,2,93,21.12.2016 03:05:49,2016-12-20 18:05:49,2016-12-21 03:07:22,2016-12-20 18:07:22,...,NA,200116,NA,,NA,NA,NA,,2,4
2340275,0,0,2,1,67,21.12.2016 03:08:00,2016-12-20 18:08:00,2016-12-21 03:09:11,2016-12-20 18:09:11,...,200352,NA,NA,,NA,NA,NA,,0,0


## 4. Activity 생성
### 사실 여기부터는 모든 것이 가정임

In [45]:
activity <- select(hans, CALLID, ACWTIME, ANSHOLDTIME, CONSULTTIME, DISPTIME, SEGSTART_UTC, SEGSTOP_UTC, 
                   TALKTIME, ORIGHOLDTIME, QUEUETIME, RINGTIME, ORIG_LOCID, CALL_DISP, DURATION)
head(activity)

CALLID,ACWTIME,ANSHOLDTIME,CONSULTTIME,DISPTIME,SEGSTART_UTC,SEGSTOP_UTC,TALKTIME,ORIGHOLDTIME,QUEUETIME,RINGTIME,ORIG_LOCID,CALL_DISP,DURATION
2340247,0,15,0,21,2016-12-20 17:54:52,2016-12-20 18:05:21,593,0,0,0,0,2,629
2340254,0,0,0,2,2016-12-20 18:04:59,2016-12-20 18:06:32,91,0,0,0,0,2,93
2340255,0,0,2,1,2016-12-20 18:06:18,2016-12-20 18:06:49,0,0,0,0,11,1,31
2340259,1,11,0,8,2016-12-20 18:04:25,2016-12-20 18:07:12,147,0,6,0,0,2,122
2340260,0,0,0,2,2016-12-20 18:05:49,2016-12-20 18:07:22,91,0,0,0,0,2,93
2340275,0,0,2,1,2016-12-20 18:08:00,2016-12-20 18:09:11,0,0,0,0,11,1,67


In [46]:
activity$ACWTIME2 <- activity$ACWTIME
head(activity, 1)

CALLID,ACWTIME,ANSHOLDTIME,CONSULTTIME,DISPTIME,SEGSTART_UTC,SEGSTOP_UTC,TALKTIME,ORIGHOLDTIME,QUEUETIME,RINGTIME,ORIG_LOCID,CALL_DISP,DURATION,ACWTIME2
2340247,0,15,0,21,2016-12-20 17:54:52,2016-12-20 18:05:21,593,0,0,0,0,2,629,0


In [47]:
class(activity$ACWTIME2)

[1] "integer"

### 1) 시간값 연산을 위한 형변환

In [48]:
activity$SEGSTART_UTC <- as.POSIXlt(activity$SEGSTART_UTC)
activity$SEGSTOP_UTC <- as.POSIXlt(activity$SEGSTOP_UTC)

### 2) SEGSTART_UTC에서 빼는 방식으로 RINGTIME, QUEUETIME 연산

In [49]:
activity$QUEUETIME <- activity$SEGSTART_UTC - activity$QUEUETIME
activity$RINGTIME <- activity$QUEUETIME - activity$RINGTIME

### 3) SEGSTART_UTC에서 더하는 방식으로 나머지 시간값을 계산하기 위해 새로운 Column 생성

In [50]:
activity$D <- activity$DISPTIME
activity$C <- activity$CONSULTTIME
activity$A <- activity$ANSHOLDTIME
activity$TT <- activity$TALKTIME

### 4) 더하기 방식으로 시간 계산

In [51]:
activity$DISPTIME <- activity$SEGSTART_UTC
activity$CONSULTTIME <- activity$DISPTIME + activity$D
activity$ANSHOLDTIME <- activity$CONSULTTIME + activity$C
activity$TALKTIME <- activity$ANSHOLDTIME + activity$A
activity$ACWTIME <- activity$TALKTIME + activity$TT
activity$ACWTIMW3 <- activity$ACWTIME + activity$ACWTIME2

In [52]:
head(activity)

CALLID,ACWTIME,ANSHOLDTIME,CONSULTTIME,DISPTIME,SEGSTART_UTC,SEGSTOP_UTC,TALKTIME,ORIGHOLDTIME,QUEUETIME,RINGTIME,ORIG_LOCID,CALL_DISP,DURATION,ACWTIME2,D,C,A,TT,ACWTIMW3
2340247,2016-12-20 18:05:21,2016-12-20 17:55:13,2016-12-20 17:55:13,2016-12-20 17:54:52,2016-12-20 17:54:52,2016-12-20 18:05:21,2016-12-20 17:55:28,0,2016-12-20 17:54:52,2016-12-20 17:54:52,0,2,629,0,21,0,15,593,2016-12-20 18:05:21
2340254,2016-12-20 18:06:32,2016-12-20 18:05:01,2016-12-20 18:05:01,2016-12-20 18:04:59,2016-12-20 18:04:59,2016-12-20 18:06:32,2016-12-20 18:05:01,0,2016-12-20 18:04:59,2016-12-20 18:04:59,0,2,93,0,2,0,0,91,2016-12-20 18:06:32
2340255,2016-12-20 18:06:21,2016-12-20 18:06:21,2016-12-20 18:06:19,2016-12-20 18:06:18,2016-12-20 18:06:18,2016-12-20 18:06:49,2016-12-20 18:06:21,0,2016-12-20 18:06:18,2016-12-20 18:06:18,11,1,31,0,1,2,0,0,2016-12-20 18:06:21
2340259,2016-12-20 18:07:11,2016-12-20 18:04:33,2016-12-20 18:04:33,2016-12-20 18:04:25,2016-12-20 18:04:25,2016-12-20 18:07:12,2016-12-20 18:04:44,0,2016-12-20 18:04:19,2016-12-20 18:04:19,0,2,122,1,8,0,11,147,2016-12-20 18:07:12
2340260,2016-12-20 18:07:22,2016-12-20 18:05:51,2016-12-20 18:05:51,2016-12-20 18:05:49,2016-12-20 18:05:49,2016-12-20 18:07:22,2016-12-20 18:05:51,0,2016-12-20 18:05:49,2016-12-20 18:05:49,0,2,93,0,2,0,0,91,2016-12-20 18:07:22
2340275,2016-12-20 18:08:03,2016-12-20 18:08:03,2016-12-20 18:08:01,2016-12-20 18:08:00,2016-12-20 18:08:00,2016-12-20 18:09:11,2016-12-20 18:08:03,0,2016-12-20 18:08:00,2016-12-20 18:08:00,11,1,67,0,1,2,0,0,2016-12-20 18:08:03


### 5) 시간 값 제외하고 날리기

In [53]:
activity2 <- select(activity, CALLID, ACWTIME, ANSHOLDTIME, CONSULTTIME, DISPTIME, SEGSTART_UTC, SEGSTOP_UTC, TALKTIME, QUEUETIME, RINGTIME, ACWTIME2, ACWTIMW3)
head(activity2)

CALLID,ACWTIME,ANSHOLDTIME,CONSULTTIME,DISPTIME,SEGSTART_UTC,SEGSTOP_UTC,TALKTIME,QUEUETIME,RINGTIME,ACWTIME2,ACWTIMW3
2340247,2016-12-20 18:05:21,2016-12-20 17:55:13,2016-12-20 17:55:13,2016-12-20 17:54:52,2016-12-20 17:54:52,2016-12-20 18:05:21,2016-12-20 17:55:28,2016-12-20 17:54:52,2016-12-20 17:54:52,0,2016-12-20 18:05:21
2340254,2016-12-20 18:06:32,2016-12-20 18:05:01,2016-12-20 18:05:01,2016-12-20 18:04:59,2016-12-20 18:04:59,2016-12-20 18:06:32,2016-12-20 18:05:01,2016-12-20 18:04:59,2016-12-20 18:04:59,0,2016-12-20 18:06:32
2340255,2016-12-20 18:06:21,2016-12-20 18:06:21,2016-12-20 18:06:19,2016-12-20 18:06:18,2016-12-20 18:06:18,2016-12-20 18:06:49,2016-12-20 18:06:21,2016-12-20 18:06:18,2016-12-20 18:06:18,0,2016-12-20 18:06:21
2340259,2016-12-20 18:07:11,2016-12-20 18:04:33,2016-12-20 18:04:33,2016-12-20 18:04:25,2016-12-20 18:04:25,2016-12-20 18:07:12,2016-12-20 18:04:44,2016-12-20 18:04:19,2016-12-20 18:04:19,1,2016-12-20 18:07:12
2340260,2016-12-20 18:07:22,2016-12-20 18:05:51,2016-12-20 18:05:51,2016-12-20 18:05:49,2016-12-20 18:05:49,2016-12-20 18:07:22,2016-12-20 18:05:51,2016-12-20 18:05:49,2016-12-20 18:05:49,0,2016-12-20 18:07:22
2340275,2016-12-20 18:08:03,2016-12-20 18:08:03,2016-12-20 18:08:01,2016-12-20 18:08:00,2016-12-20 18:08:00,2016-12-20 18:09:11,2016-12-20 18:08:03,2016-12-20 18:08:00,2016-12-20 18:08:00,0,2016-12-20 18:08:03


In [54]:
activity2 <- activity2[, c(1, 10, 9, 5, 4, 3, 8, 2, 6, 7, 11, 12)]
head(activity2)

CALLID,RINGTIME,QUEUETIME,DISPTIME,CONSULTTIME,ANSHOLDTIME,TALKTIME,ACWTIME,SEGSTART_UTC,SEGSTOP_UTC,ACWTIME2,ACWTIMW3
2340247,2016-12-20 17:54:52,2016-12-20 17:54:52,2016-12-20 17:54:52,2016-12-20 17:55:13,2016-12-20 17:55:13,2016-12-20 17:55:28,2016-12-20 18:05:21,2016-12-20 17:54:52,2016-12-20 18:05:21,0,2016-12-20 18:05:21
2340254,2016-12-20 18:04:59,2016-12-20 18:04:59,2016-12-20 18:04:59,2016-12-20 18:05:01,2016-12-20 18:05:01,2016-12-20 18:05:01,2016-12-20 18:06:32,2016-12-20 18:04:59,2016-12-20 18:06:32,0,2016-12-20 18:06:32
2340255,2016-12-20 18:06:18,2016-12-20 18:06:18,2016-12-20 18:06:18,2016-12-20 18:06:19,2016-12-20 18:06:21,2016-12-20 18:06:21,2016-12-20 18:06:21,2016-12-20 18:06:18,2016-12-20 18:06:49,0,2016-12-20 18:06:21
2340259,2016-12-20 18:04:19,2016-12-20 18:04:19,2016-12-20 18:04:25,2016-12-20 18:04:33,2016-12-20 18:04:33,2016-12-20 18:04:44,2016-12-20 18:07:11,2016-12-20 18:04:25,2016-12-20 18:07:12,1,2016-12-20 18:07:12
2340260,2016-12-20 18:05:49,2016-12-20 18:05:49,2016-12-20 18:05:49,2016-12-20 18:05:51,2016-12-20 18:05:51,2016-12-20 18:05:51,2016-12-20 18:07:22,2016-12-20 18:05:49,2016-12-20 18:07:22,0,2016-12-20 18:07:22
2340275,2016-12-20 18:08:00,2016-12-20 18:08:00,2016-12-20 18:08:00,2016-12-20 18:08:01,2016-12-20 18:08:03,2016-12-20 18:08:03,2016-12-20 18:08:03,2016-12-20 18:08:00,2016-12-20 18:09:11,0,2016-12-20 18:08:03


## 5. Activity Time이 0초인 경우 제외
### Activity Time이 0인 경우 해당 Activity가 발생하지 않았다고 가정

In [55]:
activity2$CALLID <- as.character(activity2$CALLID)
activity2$ACWTIME <- as.character(activity2$ACWTIME)
activity2$ANSHOLDTIME <- as.character(activity2$ANSHOLDTIME)
activity2$CONSULTTIME <- as.character(activity2$CONSULTTIME)
activity2$DISPTIME <- as.character(activity2$DISPTIME)
activity2$SEGSTART_UTC <- as.character(activity2$SEGSTART_UTC)
activity2$SEGSTOP_UTC <- as.character(activity2$SEGSTOP_UTC)
activity2$TALKTIME <- as.character(activity2$TALKTIME)
activity2$QUEUETIME <- as.character(activity2$QUEUETIME)
activity2$RINGTIME <- as.character(activity2$RINGTIME)
activity2$ACWTIMW3 <- as.character(activity2$ACWTIMW3)

In [56]:
activity2$RINGTIME <- ifelse(activity2$RINGTIME == activity2$QUEUETIME, "0", activity2$RINGTIME)
activity2$QUEUETIME <- ifelse(activity2$QUEUETIME == activity2$DISPTIME, "0", activity2$QUEUETIME)
activity2$DISPTIME <- ifelse(activity2$DISPTIME == activity2$CONSULTTIME, "0", activity2$DISPTIME)
activity2$CONSULTTIME <- ifelse(activity2$CONSULTTIME == activity2$ANSHOLDTIME, "0", activity2$CONSULTTIME)
activity2$ANSHOLDTIME <- ifelse(activity2$ANSHOLDTIME == activity2$TALKTIME, "0", activity2$ANSHOLDTIME)
activity2$TALKTIME <- ifelse(activity2$TALKTIME == activity2$ACWTIME, "0", activity2$TALKTIME)
activity2$ACWTIME <- ifelse(activity2$ACWTIME2 == 0, "0", activity2$ACWTIME)
activity2$ACWTIMW3 <- ifelse(activity2$ACWTIME2 == 0, "0", activity2$ACWTIMW3)

head(activity2, 20)

CALLID,RINGTIME,QUEUETIME,DISPTIME,CONSULTTIME,ANSHOLDTIME,TALKTIME,ACWTIME,SEGSTART_UTC,SEGSTOP_UTC,ACWTIME2,ACWTIMW3
2340247,0,0,2016-12-20 17:54:52,0,2016-12-20 17:55:13,2016-12-20 17:55:28,0,2016-12-20 17:54:52,2016-12-20 18:05:21,0,0
2340254,0,0,2016-12-20 18:04:59,0,0,2016-12-20 18:05:01,0,2016-12-20 18:04:59,2016-12-20 18:06:32,0,0
2340255,0,0,2016-12-20 18:06:18,2016-12-20 18:06:19,0,0,0,2016-12-20 18:06:18,2016-12-20 18:06:49,0,0
2340259,0,2016-12-20 18:04:19,2016-12-20 18:04:25,0,2016-12-20 18:04:33,2016-12-20 18:04:44,2016-12-20 18:07:11,2016-12-20 18:04:25,2016-12-20 18:07:12,1,2016-12-20 18:07:12
2340260,0,0,2016-12-20 18:05:49,0,0,2016-12-20 18:05:51,0,2016-12-20 18:05:49,2016-12-20 18:07:22,0,0
2340275,0,0,2016-12-20 18:08:00,2016-12-20 18:08:01,0,0,0,2016-12-20 18:08:00,2016-12-20 18:09:11,0,0
2340278,0,0,2016-12-20 18:08:52,2016-12-20 18:08:53,0,0,0,2016-12-20 18:08:52,2016-12-20 18:09:27,0,0
2340197,0,0,2016-12-20 17:57:11,0,0,2016-12-20 17:57:13,2016-12-20 18:01:47,2016-12-20 17:57:11,2016-12-20 18:01:50,3,2016-12-20 18:01:50
2340199,0,2016-12-20 17:56:25,2016-12-20 17:58:03,0,0,2016-12-20 17:59:43,0,2016-12-20 17:58:03,2016-12-20 18:01:53,0,0
2340201,0,0,2016-12-20 17:58:02,0,2016-12-20 17:58:04,2016-12-20 17:58:26,2016-12-20 18:01:55,2016-12-20 17:58:02,2016-12-20 18:01:56,1,2016-12-20 18:01:56


### 7) CALL_DISP는 나중에 붙이기로 하고 일단 Melt

In [57]:
activity3 <- select(activity2, -SEGSTART_UTC, -SEGSTOP_UTC)
head(activity3)

CALLID,RINGTIME,QUEUETIME,DISPTIME,CONSULTTIME,ANSHOLDTIME,TALKTIME,ACWTIME,ACWTIME2,ACWTIMW3
2340247,0,0,2016-12-20 17:54:52,0,2016-12-20 17:55:13,2016-12-20 17:55:28,0,0,0
2340254,0,0,2016-12-20 18:04:59,0,0,2016-12-20 18:05:01,0,0,0
2340255,0,0,2016-12-20 18:06:18,2016-12-20 18:06:19,0,0,0,0,0
2340259,0,2016-12-20 18:04:19,2016-12-20 18:04:25,0,2016-12-20 18:04:33,2016-12-20 18:04:44,2016-12-20 18:07:11,1,2016-12-20 18:07:12
2340260,0,0,2016-12-20 18:05:49,0,0,2016-12-20 18:05:51,0,0,0
2340275,0,0,2016-12-20 18:08:00,2016-12-20 18:08:01,0,0,0,0,0


In [58]:
activity4 <- select(activity3, -ACWTIME2)
head(activity4)

CALLID,RINGTIME,QUEUETIME,DISPTIME,CONSULTTIME,ANSHOLDTIME,TALKTIME,ACWTIME,ACWTIMW3
2340247,0,0,2016-12-20 17:54:52,0,2016-12-20 17:55:13,2016-12-20 17:55:28,0,0
2340254,0,0,2016-12-20 18:04:59,0,0,2016-12-20 18:05:01,0,0
2340255,0,0,2016-12-20 18:06:18,2016-12-20 18:06:19,0,0,0,0
2340259,0,2016-12-20 18:04:19,2016-12-20 18:04:25,0,2016-12-20 18:04:33,2016-12-20 18:04:44,2016-12-20 18:07:11,2016-12-20 18:07:12
2340260,0,0,2016-12-20 18:05:49,0,0,2016-12-20 18:05:51,0,0
2340275,0,0,2016-12-20 18:08:00,2016-12-20 18:08:01,0,0,0,0


In [59]:
actmelt <- as.data.frame(melt(activity4, id.vars = 1))
str(actmelt)

'data.frame':	6989888 obs. of  3 variables:
 $ CALLID  : chr  "2340247" "2340254" "2340255" "2340259" ...
 $ variable: Factor w/ 8 levels "RINGTIME","QUEUETIME",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ value   : chr  "0" "0" "0" "0" ...


In [60]:
colnames(actmelt) <- c("CALLID", "ACTIVITY", "TIMESTAMP")
head(actmelt)

CALLID,ACTIVITY,TIMESTAMP
2340247,RINGTIME,0
2340254,RINGTIME,0
2340255,RINGTIME,0
2340259,RINGTIME,0
2340260,RINGTIME,0
2340275,RINGTIME,0


## 5. actmelt에 CALL_DISP 붙이기

In [61]:
actcall <- actmelt[actmelt$ACTIVITY == "DISPTIME", ]
str(actcall)

'data.frame':	873736 obs. of  3 variables:
 $ CALLID   : chr  "2340247" "2340254" "2340255" "2340259" ...
 $ ACTIVITY : Factor w/ 8 levels "RINGTIME","QUEUETIME",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ TIMESTAMP: chr  "2016-12-20 17:54:52" "2016-12-20 18:04:59" "2016-12-20 18:06:18" "2016-12-20 18:04:25" ...


In [63]:
dispcall <- select(hans, CALLID, CALL_DISP)
head(dispcall)

CALLID,CALL_DISP
2340247,2
2340254,2
2340255,1
2340259,2
2340260,2
2340275,1


In [64]:
dispcall$CALLID <- as.character(dispcall$CALLID)
meltdisp <- left_join(actcall, dispcall, by = c("CALLID" = "CALLID"))
head(meltdisp)

CALLID,ACTIVITY,TIMESTAMP,CALL_DISP
2340247,DISPTIME,2016-12-20 17:54:52,2
2340254,DISPTIME,2016-12-20 18:04:59,2
2340255,DISPTIME,2016-12-20 18:06:18,1
2340259,DISPTIME,2016-12-20 18:04:25,2
2340260,DISPTIME,2016-12-20 18:05:49,2
2340275,DISPTIME,2016-12-20 18:08:00,1


In [65]:
meltdisp$DISPNAME <- "a"
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 1, "DISP_CONN", meltdisp$DISPNAME)
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 2, "DISP_ANS", meltdisp$DISPNAME)
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 3, "DISP_ABAN", meltdisp$DISPNAME)
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 4, "DISP_IFLOW", meltdisp$DISPNAME)
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 5, "DISP_FBUSY", meltdisp$DISPNAME)
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 6, "DISP_FDISC", meltdisp$DISPNAME)
meltdisp$DISPNAME <- ifelse(meltdisp$CALL_DISP == 7, "DISP_OTHER", meltdisp$DISPNAME)
head(meltdisp)

CALLID,ACTIVITY,TIMESTAMP,CALL_DISP,DISPNAME
2340247,DISPTIME,2016-12-20 17:54:52,2,DISP_ANS
2340254,DISPTIME,2016-12-20 18:04:59,2,DISP_ANS
2340255,DISPTIME,2016-12-20 18:06:18,1,DISP_CONN
2340259,DISPTIME,2016-12-20 18:04:25,2,DISP_ANS
2340260,DISPTIME,2016-12-20 18:05:49,2,DISP_ANS
2340275,DISPTIME,2016-12-20 18:08:00,1,DISP_CONN


In [66]:
table(meltdisp$DISPNAME)
table(meltdisp$CALL_DISP)


 DISP_ABAN   DISP_ANS  DISP_CONN DISP_FDISC DISP_OTHER 
    119851     205012     478671       4987      65215 


     1      2      3      6      7 
478671 205012 119851   4987  65215 

In [67]:
meltdisp$ACTIVITY <- meltdisp$DISPNAME
meltdisp <- select(meltdisp, -CALL_DISP, -DISPNAME)
head(meltdisp)

CALLID,ACTIVITY,TIMESTAMP
2340247,DISP_ANS,2016-12-20 17:54:52
2340254,DISP_ANS,2016-12-20 18:04:59
2340255,DISP_CONN,2016-12-20 18:06:18
2340259,DISP_ANS,2016-12-20 18:04:25
2340260,DISP_ANS,2016-12-20 18:05:49
2340275,DISP_CONN,2016-12-20 18:08:00


In [68]:
actmelt2 <- actmelt[actmelt$ACTIVITY != "DISPTIME", ]
table(actmelt2$ACTIVITY)


   RINGTIME   QUEUETIME    DISPTIME CONSULTTIME ANSHOLDTIME    TALKTIME 
     873736      873736           0      873736      873736      873736 
    ACWTIME    ACWTIMW3 
     873736      873736 

In [69]:
actmelt2 <- rbind(meltdisp, actmelt2)
str(actmelt2)

'data.frame':	6989888 obs. of  3 variables:
 $ CALLID   : chr  "2340247" "2340254" "2340255" "2340259" ...
 $ ACTIVITY : chr  "DISP_ANS" "DISP_ANS" "DISP_CONN" "DISP_ANS" ...
 $ TIMESTAMP: chr  "2016-12-20 17:54:52" "2016-12-20 18:04:59" "2016-12-20 18:06:18" "2016-12-20 18:04:25" ...


In [70]:
table(actmelt2$ACTIVITY)
actmelt3 <- actmelt2


    ACWTIME    ACWTIMW3 ANSHOLDTIME CONSULTTIME   DISP_ABAN    DISP_ANS 
     873736      873736      873736      873736      119851      205012 
  DISP_CONN  DISP_FDISC  DISP_OTHER   QUEUETIME    RINGTIME    TALKTIME 
     478671        4987       65215      873736      873736      873736 

## 7. 상담원 ID를 넣어보자

In [72]:
marketer <- select(hans, CALLID, ORIGLOGIN, ANSLOGIN)
head(marketer, 20)

CALLID,ORIGLOGIN,ANSLOGIN
2340247,NA,200508
2340254,NA,200591
2340255,200431,NA
2340259,NA,200331
2340260,NA,200116
2340275,200352,NA
2340278,200119,NA
2340197,NA,200312
2340199,NA,200118
2340201,NA,200119


In [73]:
table(is.na(marketer$ORIGLOGIN) == TRUE)
table(is.na(marketer$ANSLOGIN) == TRUE)


 FALSE   TRUE 
121554 752182 


 FALSE   TRUE 
205218 668518 

In [74]:
marketer$LOGID <- ifelse(is.na(marketer$ORIGLOGIN) == TRUE, marketer$ANSLOGIN, marketer$ORIGLOGIN)
table(is.na(marketer$LOGID) == TRUE)


 FALSE   TRUE 
326744 546992 

In [75]:
head(actmelt3)

CALLID,ACTIVITY,TIMESTAMP
2340247,DISP_ANS,2016-12-20 17:54:52
2340254,DISP_ANS,2016-12-20 18:04:59
2340255,DISP_CONN,2016-12-20 18:06:18
2340259,DISP_ANS,2016-12-20 18:04:25
2340260,DISP_ANS,2016-12-20 18:05:49
2340275,DISP_CONN,2016-12-20 18:08:00


In [76]:
marketer$CALLID <- as.character(marketer$CALLID)
actmelt4 <- left_join(actmelt3, marketer, by = c("CALLID" = "CALLID"))
head(actmelt4)

CALLID,ACTIVITY,TIMESTAMP,ORIGLOGIN,ANSLOGIN,LOGID
2340247,DISP_ANS,2016-12-20 17:54:52,NA,200508,200508
2340254,DISP_ANS,2016-12-20 18:04:59,NA,200591,200591
2340255,DISP_CONN,2016-12-20 18:06:18,200431,NA,200431
2340259,DISP_ANS,2016-12-20 18:04:25,NA,200331,200331
2340260,DISP_ANS,2016-12-20 18:05:49,NA,200116,200116
2340275,DISP_CONN,2016-12-20 18:08:00,200352,NA,200352


## 8. LOCID를 넣어보자

In [78]:
locid <- select(hans, CALLID, EQ_LOCID, ORIG_LOCID, ANS_LOCID, OBS_LOCID)
head(locid)

CALLID,EQ_LOCID,ORIG_LOCID,ANS_LOCID,OBS_LOCID
2340247,1,0,11,0
2340254,0,0,11,0
2340255,0,11,0,0
2340259,0,0,11,0
2340260,0,0,11,0
2340275,0,11,0,0


In [79]:
table(locid$EQ_LOCID == 0)
table(locid$ORIG_LOCID == 0)
table(locid$ANS_LOCID == 0)
table(locid$OBS_LOCID == 0)


 FALSE   TRUE 
373050 500686 


 FALSE   TRUE 
121554 752182 


 FALSE   TRUE 
205218 668518 


 FALSE   TRUE 
    43 873693 

In [67]:
## NULL이 너무 많아서 의미가 없을 것 같음

## 9. ~~ 여부를 넣어 보자

In [82]:
flag <- select(hans, CALLID, UUI_LEN, ASSIST, AUDIO, CONFERENCE, DA_QUEUED, HOLDABN, MALICIOUS, OBSERVINGCALL, TRANSFERRED, AGT_RELEASED)
head(flag)

CALLID,UUI_LEN,ASSIST,AUDIO,CONFERENCE,DA_QUEUED,HOLDABN,MALICIOUS,OBSERVINGCALL,TRANSFERRED,AGT_RELEASED
2340247,0,0,0,1,0,1,0,0,0,0
2340254,0,0,0,0,0,0,0,0,0,0
2340255,0,0,0,0,0,0,0,0,0,0
2340259,0,0,0,1,0,0,0,0,0,1
2340260,0,0,0,0,0,0,0,0,0,0
2340275,0,0,0,0,0,0,0,0,0,0


In [83]:
flag$CALLID <- as.character(flag$CALLID)
actmelt5 <- left_join(actmelt4, flag, by = c("CALLID" = "CALLID"))
head(actmelt5)

actmelt5 <- select(actmelt5, -UUI_LEN)
head(actmelt5)

CALLID,ACTIVITY,TIMESTAMP,ORIGLOGIN,ANSLOGIN,LOGID,UUI_LEN,ASSIST,AUDIO,CONFERENCE,DA_QUEUED,HOLDABN,MALICIOUS,OBSERVINGCALL,TRANSFERRED,AGT_RELEASED
2340247,DISP_ANS,2016-12-20 17:54:52,NA,200508,200508,0,0,0,1,0,1,0,0,0,0
2340254,DISP_ANS,2016-12-20 18:04:59,NA,200591,200591,0,0,0,0,0,0,0,0,0,0
2340255,DISP_CONN,2016-12-20 18:06:18,200431,NA,200431,0,0,0,0,0,0,0,0,0,0
2340259,DISP_ANS,2016-12-20 18:04:25,NA,200331,200331,0,0,0,1,0,0,0,0,0,1
2340260,DISP_ANS,2016-12-20 18:05:49,NA,200116,200116,0,0,0,0,0,0,0,0,0,0
2340275,DISP_CONN,2016-12-20 18:08:00,200352,NA,200352,0,0,0,0,0,0,0,0,0,0


CALLID,ACTIVITY,TIMESTAMP,ORIGLOGIN,ANSLOGIN,LOGID,ASSIST,AUDIO,CONFERENCE,DA_QUEUED,HOLDABN,MALICIOUS,OBSERVINGCALL,TRANSFERRED,AGT_RELEASED
2340247,DISP_ANS,2016-12-20 17:54:52,NA,200508,200508,0,0,1,0,1,0,0,0,0
2340254,DISP_ANS,2016-12-20 18:04:59,NA,200591,200591,0,0,0,0,0,0,0,0,0
2340255,DISP_CONN,2016-12-20 18:06:18,200431,NA,200431,0,0,0,0,0,0,0,0,0
2340259,DISP_ANS,2016-12-20 18:04:25,NA,200331,200331,0,0,1,0,0,0,0,0,1
2340260,DISP_ANS,2016-12-20 18:05:49,NA,200116,200116,0,0,0,0,0,0,0,0,0
2340275,DISP_CONN,2016-12-20 18:08:00,200352,NA,200352,0,0,0,0,0,0,0,0,0


## 10. 한국말로 바꿔보자

In [84]:
colnames(actmelt5) <- c("CALLID", "ACTIVITY", "TIMESTAMP", "발신상담원", "응답상담원", "발신응답상담원", "보조요청여부", "난청여부", "컨퍼런스여부", "직접상담원통화대기여부", "보류중포기여부", "불량통화추적활성화여부", "감청여부", "호전환여부", "상담원에의한종료여부")

In [85]:
head(actmelt5)

CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
2340247,DISP_ANS,2016-12-20 17:54:52,NA,200508,200508,0,0,1,0,1,0,0,0,0
2340254,DISP_ANS,2016-12-20 18:04:59,NA,200591,200591,0,0,0,0,0,0,0,0,0
2340255,DISP_CONN,2016-12-20 18:06:18,200431,NA,200431,0,0,0,0,0,0,0,0,0
2340259,DISP_ANS,2016-12-20 18:04:25,NA,200331,200331,0,0,1,0,0,0,0,0,1
2340260,DISP_ANS,2016-12-20 18:05:49,NA,200116,200116,0,0,0,0,0,0,0,0,0
2340275,DISP_CONN,2016-12-20 18:08:00,200352,NA,200352,0,0,0,0,0,0,0,0,0


In [86]:
table(actmelt5$ACTIVITY)
class(actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- as.character(actmelt5$ACTIVITY)
class(actmelt5$ACTIVITY)


    ACWTIME    ACWTIMW3 ANSHOLDTIME CONSULTTIME   DISP_ABAN    DISP_ANS 
     873736      873736      873736      873736      119851      205012 
  DISP_CONN  DISP_FDISC  DISP_OTHER   QUEUETIME    RINGTIME    TALKTIME 
     478671        4987       65215      873736      873736      873736 

[1] "character"

[1] "character"

In [87]:
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "ACWTIME", "후처리", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "ANSHOLDTIME", "응대중보류", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "CONSULTTIME", "컨설팅", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "DISP_ABAN", "콜처리_포기", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "DISP_ANS", "콜처리_응답", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "DISP_CONN", "콜처리_연결", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "DISP_FDISC", "콜처리_강제종료", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "DISP_OTHER", "기타", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "QUEUETIME", "큐대기", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "RINGTIME", "벨울림", actmelt5$ACTIVITY)
actmelt5$ACTIVITY <- ifelse(actmelt5$ACTIVITY == "TALKTIME", "응대", actmelt5$ACTIVITY)

head(actmelt5, 12)

CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
2340247,콜처리_응답,2016-12-20 17:54:52,NA,200508,200508,0,0,1,0,1,0,0,0,0
2340254,콜처리_응답,2016-12-20 18:04:59,NA,200591,200591,0,0,0,0,0,0,0,0,0
2340255,콜처리_연결,2016-12-20 18:06:18,200431,NA,200431,0,0,0,0,0,0,0,0,0
2340259,콜처리_응답,2016-12-20 18:04:25,NA,200331,200331,0,0,1,0,0,0,0,0,1
2340260,콜처리_응답,2016-12-20 18:05:49,NA,200116,200116,0,0,0,0,0,0,0,0,0
2340275,콜처리_연결,2016-12-20 18:08:00,200352,NA,200352,0,0,0,0,0,0,0,0,0
2340278,콜처리_연결,2016-12-20 18:08:52,200119,NA,200119,0,0,0,0,0,0,0,0,0
2340197,콜처리_응답,2016-12-20 17:57:11,NA,200312,200312,0,0,0,0,0,0,0,0,0
2340199,콜처리_응답,2016-12-20 17:58:03,NA,200118,200118,0,0,0,0,0,0,0,0,0
2340201,콜처리_응답,2016-12-20 17:58:02,NA,200119,200119,0,0,1,0,0,0,0,0,1


In [88]:
actmelt6 <- actmelt5
table(actmelt6$TIMESTAMP == "0")


  FALSE    TRUE 
1863879 5126009 

In [89]:
actmelt6$TIMESTAMP <- ifelse(actmelt6$TIMESTAMP == "0", NA, actmelt6$TIMESTAMP)
tail(actmelt6, 20)

,CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
6989869,3203943,ACWTIMW3,NA,200558,NA,200558,0,0,0,0,0,0,0,0,0
6989870,3204252,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
6989871,3204253,ACWTIMW3,NA,200238,NA,200238,0,0,0,0,0,0,0,0,0
6989872,3204254,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
6989873,3204255,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
6989874,3204256,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
6989875,3204257,ACWTIMW3,2016-12-26 07:09:15,NA,200343,200343,0,0,0,0,0,0,0,0,1
6989876,3204259,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
6989877,3204260,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
6989878,3204262,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0


In [90]:
actmelt6 <- arrange(actmelt6, CALLID, TIMESTAMP)

In [91]:
head(actmelt6, 20)

CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
2340195,콜처리_연결,2016-12-20 18:00:16,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,벨울림,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,큐대기,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,컨설팅,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,응대중보류,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,응대,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,후처리,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340195,ACWTIMW3,NA,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340196,큐대기,2016-12-20 18:01:23,NA,NA,NA,0,0,0,0,0,0,0,0,0
2340196,콜처리_연결,2016-12-20 18:01:24,NA,NA,NA,0,0,0,0,0,0,0,0,0


In [92]:
actmelt6$발신상담원 <- ifelse(is.na(actmelt6$발신상담원) == TRUE, "0", actmelt6$발신상담원)
actmelt6$응답상담원 <- ifelse(is.na(actmelt6$응답상담원) == TRUE, "0", actmelt6$응답상담원)
actmelt6$발신응답상담원 <- ifelse(is.na(actmelt6$발신응답상담원) == TRUE, "0", actmelt6$발신응답상담원)

head(actmelt6, 20)

CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
2340195,콜처리_연결,2016-12-20 18:00:16,0,0,0,0,0,0,0,0,0,0,0,0
2340195,벨울림,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340195,큐대기,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340195,컨설팅,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340195,응대중보류,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340195,응대,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340195,후처리,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340195,ACWTIMW3,NA,0,0,0,0,0,0,0,0,0,0,0,0
2340196,큐대기,2016-12-20 18:01:23,0,0,0,0,0,0,0,0,0,0,0,0
2340196,콜처리_연결,2016-12-20 18:01:24,0,0,0,0,0,0,0,0,0,0,0,0


In [93]:
actmelt7 <- na.omit(actmelt6)
str(actmelt7)

'data.frame':	1863879 obs. of  15 variables:
 $ CALLID                : chr  "2340195" "2340196" "2340196" "2340197" ...
 $ ACTIVITY              : chr  "콜처리_연결" "큐대기" "콜처리_연결" "콜처리_응답" ...
 $ TIMESTAMP             : chr  "2016-12-20 18:00:16" "2016-12-20 18:01:23" "2016-12-20 18:01:24" "2016-12-20 17:57:11" ...
 $ 발신상담원            : chr  "0" "0" "0" "0" ...
 $ 응답상담원            : chr  "0" "0" "0" "200312" ...
 $ 발신응답상담원        : chr  "0" "0" "0" "200312" ...
 $ 보조요청여부          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 난청여부              : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 컨퍼런스여부          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 직접상담원통화대기여부: int  0 0 0 0 0 0 0 0 0 0 ...
 $ 보류중포기여부        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 불량통화추적활성화여부: int  0 0 0 0 0 0 0 0 0 0 ...
 $ 감청여부              : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 호전환여부            : int  0 0 0 0 0 0 0 0 0 0 ...
 $ 상담원에의한종료여부  : int  0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:5126009] 2 3 4 5 6 7 8 11 12 13 ...
  .. ..- a

In [94]:
head(actmelt7, 30)

,CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
1,2340195,콜처리_연결,2016-12-20 18:00:16,0,0,0,0,0,0,0,0,0,0,0,0
9,2340196,큐대기,2016-12-20 18:01:23,0,0,0,0,0,0,0,0,0,0,0,0
10,2340196,콜처리_연결,2016-12-20 18:01:24,0,0,0,0,0,0,0,0,0,0,0,0
17,2340197,콜처리_응답,2016-12-20 17:57:11,0,200312,200312,0,0,0,0,0,0,0,0,0
18,2340197,응대,2016-12-20 17:57:13,0,200312,200312,0,0,0,0,0,0,0,0,0
19,2340197,후처리,2016-12-20 18:01:47,0,200312,200312,0,0,0,0,0,0,0,0,0
20,2340197,ACWTIMW3,2016-12-20 18:01:50,0,200312,200312,0,0,0,0,0,0,0,0,0
25,2340198,큐대기,2016-12-20 18:00:15,0,0,0,0,0,0,0,0,0,0,0,0
26,2340198,콜처리_포기,2016-12-20 18:00:55,0,0,0,0,0,0,0,0,0,0,0,0
33,2340199,큐대기,2016-12-20 17:56:25,0,200118,200118,0,0,0,0,0,0,0,0,0


In [95]:
table(actmelt7$ACTIVITY)


       ACWTIMW3          벨울림            응대      응대중보류          컨설팅 
         171855             193          204763           31650          121488 
콜처리_강제종료     콜처리_연결     콜처리_응답     콜처리_포기          큐대기 
           4987          478477          205012          117043          356556 
         후처리 
         171855 

In [104]:
actmelt8 <- actmelt7
actmelt8 <- actmelt8[actmelt8$ACTIVITY != "ACWTIMW3", ]

In [106]:
table(actmelt8$ACTIVITY)
head(actmelt8)


         벨울림            응대      응대중보류          컨설팅 콜처리_강제종료 
            193          204763           31650          121488            4987 
    콜처리_연결     콜처리_응답     콜처리_포기          큐대기          후처리 
         478477          205012          117043          356556          171855 

,CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,보조요청여부,난청여부,컨퍼런스여부,직접상담원통화대기여부,보류중포기여부,불량통화추적활성화여부,감청여부,호전환여부,상담원에의한종료여부
1,2340195,콜처리_연결,2016-12-20 18:00:16,0,0,0,0,0,0,0,0,0,0,0,0
9,2340196,큐대기,2016-12-20 18:01:23,0,0,0,0,0,0,0,0,0,0,0,0
10,2340196,콜처리_연결,2016-12-20 18:01:24,0,0,0,0,0,0,0,0,0,0,0,0
17,2340197,콜처리_응답,2016-12-20 17:57:11,0,200312,200312,0,0,0,0,0,0,0,0,0
18,2340197,응대,2016-12-20 17:57:13,0,200312,200312,0,0,0,0,0,0,0,0,0
19,2340197,후처리,2016-12-20 18:01:47,0,200312,200312,0,0,0,0,0,0,0,0,0


In [107]:
hansol3 <- actmelt8

hansol3$보조요청여부 <- as.character(hansol3$보조요청여부)
hansol3$난청여부 <- as.character(hansol3$난청여부)
hansol3$컨퍼런스여부 <- as.character(hansol3$컨퍼런스여부)
hansol3$직접상담원통화대기여부 <- as.character(hansol3$직접상담원통화대기여부)
hansol3$보류중포기여부 <- as.character(hansol3$보류중포기여부)
hansol3$불량통화추적활성화여부 <- as.character(hansol3$불량통화추적활성화여부)
hansol3$감청여부 <- as.character(hansol3$감청여부)
hansol3$호전환여부 <- as.character(hansol3$호전환여부)
hansol3$상담원에의한종료여부 <- as.character(hansol3$상담원에의한종료여부)

hansol3$보조요청여부 <- ifelse(hansol3$보조요청여부 == "1", "YES", "NO")
hansol3$난청여부 <- ifelse(hansol3$난청여부 == "1", "YES", "NO")
hansol3$컨퍼런스여부 <- ifelse(hansol3$컨퍼런스여부 == "1", "YES", "NO")
hansol3$직접상담원통화대기여부 <- ifelse(hansol3$직접상담원통화대기여부 == "1", "YES", "NO")
hansol3$보류중포기여부 <- ifelse(hansol3$보류중포기여부 == "1", "YES", "NO")
hansol3$불량통화추적활성화여부 <- ifelse(hansol3$불량통화추적활성화여부 == "1", "YES", "NO")
hansol3$감청여부 <- ifelse(hansol3$감청여부 == "1", "YES", "NO")
hansol3$호전환여부 <- ifelse(hansol3$호전환여부 == "1", "YES", "NO")
hansol3$상담원에의한종료여부 <- ifelse(hansol3$상담원에의한종료여부 == "1", "YES", "NO")

table(hansol3$보조요청여부) # all 0
table(hansol3$난청여부) # all 0
table(hansol3$컨퍼런스여부)
table(hansol3$직접상담원통화대기여부) # all 0
table(hansol3$보류중포기여부)
table(hansol3$불량통화추적활성화여부) # all 0
table(hansol3$감청여부)
table(hansol3$호전환여부)
table(hansol3$상담원에의한종료여부)


     NO 
1692024 


     NO 
1692024 


     NO     YES 
1585279  106745 


     NO 
1692024 


     NO     YES 
1690311    1713 


     NO 
1692024 


     NO     YES 
1691959      65 


     NO     YES 
1675242   16782 


     NO     YES 
1561658  130366 

In [108]:
hansol5 <- select(hansol3, -보조요청여부, -난청여부, -직접상담원통화대기여부, -불량통화추적활성화여부)
head(hansol5, 20)
table(hansol5$ACTIVITY)

,CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,컨퍼런스여부,보류중포기여부,감청여부,호전환여부,상담원에의한종료여부
1,2340195,콜처리_연결,2016-12-20 18:00:16,0,0,0,NO,NO,NO,NO,NO
9,2340196,큐대기,2016-12-20 18:01:23,0,0,0,NO,NO,NO,NO,NO
10,2340196,콜처리_연결,2016-12-20 18:01:24,0,0,0,NO,NO,NO,NO,NO
17,2340197,콜처리_응답,2016-12-20 17:57:11,0,200312,200312,NO,NO,NO,NO,NO
18,2340197,응대,2016-12-20 17:57:13,0,200312,200312,NO,NO,NO,NO,NO
19,2340197,후처리,2016-12-20 18:01:47,0,200312,200312,NO,NO,NO,NO,NO
25,2340198,큐대기,2016-12-20 18:00:15,0,0,0,NO,NO,NO,NO,NO
26,2340198,콜처리_포기,2016-12-20 18:00:55,0,0,0,NO,NO,NO,NO,NO
33,2340199,큐대기,2016-12-20 17:56:25,0,200118,200118,NO,NO,NO,NO,NO
34,2340199,콜처리_응답,2016-12-20 17:58:03,0,200118,200118,NO,NO,NO,NO,NO



         벨울림            응대      응대중보류          컨설팅 콜처리_강제종료 
            193          204763           31650          121488            4987 
    콜처리_연결     콜처리_응답     콜처리_포기          큐대기          후처리 
         478477          205012          117043          356556          171855 

In [109]:
hansol5$발신상담원 <- ifelse(hansol5$발신상담원 == "0", NA, hansol5$발신상담원)
hansol5$응답상담원 <- ifelse(hansol5$응답상담원 == "0", NA, hansol5$응답상담원)
hansol5$발신응답상담원 <- ifelse(hansol5$발신응답상담원 == "0", NA, hansol5$발신응답상담원)

head(hansol5, 20)

,CALLID,ACTIVITY,TIMESTAMP,발신상담원,응답상담원,발신응답상담원,컨퍼런스여부,보류중포기여부,감청여부,호전환여부,상담원에의한종료여부
1,2340195,콜처리_연결,2016-12-20 18:00:16,NA,NA,NA,NO,NO,NO,NO,NO
9,2340196,큐대기,2016-12-20 18:01:23,NA,NA,NA,NO,NO,NO,NO,NO
10,2340196,콜처리_연결,2016-12-20 18:01:24,NA,NA,NA,NO,NO,NO,NO,NO
17,2340197,콜처리_응답,2016-12-20 17:57:11,NA,200312,200312,NO,NO,NO,NO,NO
18,2340197,응대,2016-12-20 17:57:13,NA,200312,200312,NO,NO,NO,NO,NO
19,2340197,후처리,2016-12-20 18:01:47,NA,200312,200312,NO,NO,NO,NO,NO
25,2340198,큐대기,2016-12-20 18:00:15,NA,NA,NA,NO,NO,NO,NO,NO
26,2340198,콜처리_포기,2016-12-20 18:00:55,NA,NA,NA,NO,NO,NO,NO,NO
33,2340199,큐대기,2016-12-20 17:56:25,NA,200118,200118,NO,NO,NO,NO,NO
34,2340199,콜처리_응답,2016-12-20 17:58:03,NA,200118,200118,NO,NO,NO,NO,NO


In [110]:
# hansol5$ACTIVITY <- ifelse(hansol5$ACTIVITY == "기타", "콜처리_기타", hansol5$ACTIVITY)
## 다 날아간 것으로 추정 됨
table(hansol5$ACTIVITY)


         벨울림            응대      응대중보류          컨설팅 콜처리_강제종료 
            193          204763           31650          121488            4987 
    콜처리_연결     콜처리_응답     콜처리_포기          큐대기          후처리 
         478477          205012          117043          356556          171855 

In [111]:
write.csv(hansol5, file = "hansol_final_acwout.csv", row.names = FALSE)